In [2]:
import json
from pathlib import Path
import pandas as pd

# 1) Ruta del archivo JSON de Place Details (ajústala si es necesario)
json_path = Path("reviews.json")

# 2) Cargar y transformar el contenido JSON
with json_path.open("r", encoding="utf-8") as file:
    place_details = json.load(file)

# Explorar llaves de primer nivel
print("Llaves principales:", list(place_details.keys()))
print("status:", place_details.get("status"))

# 3) Acceder a result
result = place_details.get("result", {})
print("Campos en result:", list(result.keys())[:20])

Llaves principales: ['html_attributions', 'result', 'status']
status: OK
Campos en result: ['address_components', 'adr_address', 'business_status', 'formatted_address', 'formatted_phone_number', 'geometry', 'icon', 'international_phone_number', 'name', 'opening_hours', 'photos', 'place_id', 'plus_code', 'price_level', 'rating', 'reference', 'reviews', 'types', 'url', 'user_ratings_total']


In [4]:
# 4) Explorar detalles principales del lugar
campos_principales = {
    "business_status": result.get("business_status"),
    "formatted_address": result.get("formatted_address"),
    "geometry": result.get("geometry"),
    "name": result.get("name"),
    "price_level": result.get("price_level"),
    "rating": result.get("rating"),
    "types": result.get("types"),
    "website": result.get("website"),
}

for campo, valor in campos_principales.items():
    print(f"\n{campo}:\n{valor}")

# 5) Revisar críticas y clasificar tono
reviews = result.get("reviews", [])
reviews_df = pd.DataFrame(reviews)

if reviews_df.empty:
    print("\nNo se encontraron reviews en el JSON.")
else:
    columnas = [c for c in ["author_name", "language", "rating", "text", "relative_time_description"] if c in reviews_df.columns]
    print("\nMuestra de reviews:")
    display(reviews_df[columnas].head())

    def clasificar_sentimiento(review):
        rating = review.get("rating")
        text = str(review.get("text", "")).lower()

        positivos = ["excellent", "great", "amazing", "good", "love", "recomend", "clean", "friendly", "perfect"]
        negativos = ["bad", "terrible", "awful", "poor", "hate", "dirty", "slow", "expensive", "worst"]

        score = 0
        if isinstance(rating, (int, float)):
            if rating >= 4:
                score += 1
            elif rating <= 2:
                score -= 1

        score += sum(p in text for p in positivos)
        score -= sum(n in text for n in negativos)

        if score > 0:
            return "Positivo"
        if score < 0:
            return "Negativo"
        return "Neutro"

    reviews_df["sentimiento"] = reviews_df.apply(lambda row: clasificar_sentimiento(row.to_dict()), axis=1)

    print("\nDistribución de sentimiento:")
    display(reviews_df["sentimiento"].value_counts().rename_axis("sentimiento").reset_index(name="conteo"))

    print("\nDistribución por idioma:")
    idiomas = reviews_df["language"].value_counts().rename_axis("idioma").reset_index(name="conteo")
    display(idiomas)

    print("\nReviews clasificadas:")
    display(reviews_df[[c for c in ["author_name", "rating", "sentimiento", "text"] if c in reviews_df.columns]].head(10))


business_status:
OPERATIONAL

formatted_address:
Calle 56 47A, Zona Paseo Montejo, Centro, 97000 Mérida, Yuc., Mexico

geometry:
{'location': {'lat': 20.9798074, 'lng': -89.6183645}, 'viewport': {'northeast': {'lat': 20.9811692302915, 'lng': -89.61715956970849}, 'southwest': {'lat': 20.9784712697085, 'lng': -89.6198575302915}}}

name:
Dulcería y Sorbetería Colón

price_level:
2

rating:
4.5

types:
['food', 'point_of_interest', 'store', 'establishment']

website:
https://dulceriaysorbeteriacolon.wordpress.com/

Muestra de reviews:


,author_name,language,rating,text,relative_time_description
0,Leonard Majere,en,3,Has great reviews but...my god....this place o...,5 months ago
1,Mario Caceres,en,5,"Sorbetes, like Italian ice but much better. Al...",in the last week
2,S H,en,3,I had the famous Helado de Coco. It was good b...,4 weeks ago
3,Fernando Carvajal,en,5,The best ice cream and shakes in town love it ...,a week ago
4,C Luengas,en,4,Very good sorbets but one must eat them fast b...,2 months ago



Distribución de sentimiento:


,sentimiento,conteo
0,Positivo,5



Distribución por idioma:


,idioma,conteo
0,en,5



Reviews clasificadas:


,author_name,rating,sentimiento,text
0,Leonard Majere,3,Positivo,Has great reviews but...my god....this place o...
1,Mario Caceres,5,Positivo,"Sorbetes, like Italian ice but much better. Al..."
2,S H,3,Positivo,I had the famous Helado de Coco. It was good b...
3,Fernando Carvajal,5,Positivo,The best ice cream and shakes in town love it ...
4,C Luengas,4,Positivo,Very good sorbets but one must eat them fast b...
